In [11]:

#improt basic packages
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions
import pandas as pd
import numpy as np
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from IPython.core.interactiveshell import InteractiveShell
#from pandas_profiling import ProfileReport
#import umap

InteractiveShell.ast_node_interactivity = "all"


In [2]:
import ranger21
optimizer=ranger21()

In [ ]:
DrawingOptions.bondLineWidth=1.8
mol = Chem.MolFromSmiles('Brc1c(N)c(CNC2CCC(O)CC2)cc(Br)c1')
mol2=Chem.MolFromSmiles('C1CC(CCC1NCC2=C(C(=CC(=C2)Br)Br)N)O')
mol3=Chem.MolFromSmiles('[As](=O)(O)(O)c1ccc(NC(=O)N)cc1')
Descriptors.MolLogP(mol)
Descriptors.MolLogP(mol2)
mol
mol2
mol3
rdkit.Chem.rdMolDescriptors.BCUT2D(mol3)

In [13]:
df = pd.read_csv('/home/wzabsd/Desktop/NAIST-PBL1/data/test.csv')

#profile = ProfileReport(df, title="Pandas Profiling Report")

In [21]:
df.head()
smiles_l=df['washed_SMILES'].tolist()
#smiles_l[smiles_l.index('[i+]1c2c(c3c1cccc3)cccc2')]='C1=CC=C2C(=C1)C3=CC=CC=C3[I+]2'
#smiles_l.head()
mols_l=[Chem.MolFromSmiles(smile) for smile in smiles_l]
#Chem.MolToMolFile(mols_l, '/home/wzabsd/Desktop/NAIST-PBL1/output.mol')

,SAMPLE_ID,washed_SMILES,Class
0,NCGC00093656-03,Br/C=C/C=1C(=O)NC(=O)N([C@@H]2O[C@H](CO)[C@@H]...,Inactive
1,NCGC00262958-01,Br/C=C\C=1C(=O)NC(=O)N([C@@H]2[C@@H](O)[C@H](O...,Inactive
2,NCGC00162076-02,BrC(Br)(P(OP(=O)(OC[C@@H]1[C@@H](O)[C@@H](O)[C...,Inactive
3,NCGC00160445-02,BrC(C(=O)NC(=O)N)C(C)C,Inactive
4,NCGC00090868-06,BrC(Cl)C(F)(F)F,Inactive


In [1]:
des_l=[des_name[0] for des_name in Descriptors._descList]
desc_calc = MoleculeDescriptors.MolecularDescriptorCalculator(des_l)
descriptors = pd.DataFrame([desc_calc.CalcDescriptors(mol) for mol in mols_l])
descriptors.columns = des_l
descriptors.index = smiles_l

NameError: name 'Descriptors' is not defined

In [28]:
descriptors["SMILES"]=smiles_l
descriptors["mol"]=smiles_l
descriptors["SAMPLE_ID"]=df["SAMPLE_ID"].tolist()
descriptors["Class"]=df["Class"].tolist()


In [30]:
descriptors.head()
descriptors.to_csv("/home/wzabsd/Desktop/NAIST-PBL1/data/test.csv",index=False)

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,SMILES,mol,SAMPLE_ID,Class
Br/C=C/C=1C(=O)NC(=O)N([C@@H]2O[C@H](CO)[C@@H](O)C2)C=1,11.732651,-0.843523,11.732651,0.175242,0.694561,333.138,320.034,332.000784,104,0,...,0,0,0,0,0,0,Br/C=C/C=1C(=O)NC(=O)N([C@@H]2O[C@H](CO)[C@@H]...,Br/C=C/C=1C(=O)NC(=O)N([C@@H]2O[C@H](CO)[C@@H]...,NCGC00093656-03,Inactive
Br/C=C\C=1C(=O)NC(=O)N([C@@H]2[C@@H](O)[C@H](O)[C@@H](CO)O2)C=1,11.767095,-1.392082,11.767095,0.165938,0.527270,349.137,336.033,347.995698,110,0,...,0,0,0,0,0,0,Br/C=C\C=1C(=O)NC(=O)N([C@@H]2[C@@H](O)[C@H](O...,Br/C=C\C=1C(=O)NC(=O)N([C@@H]2[C@@H](O)[C@H](O...,NCGC00262958-01,Inactive
BrC(Br)(P(OP(=O)(OC[C@@H]1[C@@H](O)[C@@H](O)[C@H](n2c3ncnc(N(CC)CC)c3nc2)O1)[O-])(=O)[O-])P(=O)(O)[O-],12.067540,-5.950858,12.067540,0.247101,0.191820,716.086,694.918,713.878276,210,0,...,0,0,0,0,0,0,BrC(Br)(P(OP(=O)(OC[C@@H]1[C@@H](O)[C@@H](O)[C...,BrC(Br)(P(OP(=O)(OC[C@@H]1[C@@H](O)[C@@H](O)[C...,NCGC00162076-02,Inactive
BrC(C(=O)NC(=O)N)C(C)C,10.936088,-0.819352,10.936088,0.133657,0.672535,223.070,211.982,222.000390,64,0,...,0,0,0,0,0,1,BrC(C(=O)NC(=O)N)C(C)C,BrC(C(=O)NC(=O)N)C(C)C,NCGC00160445-02,Inactive
BrC(Cl)C(F)(F)F,10.999421,-4.317901,10.999421,1.916667,0.523506,197.381,196.373,195.890224,44,0,...,0,0,0,0,0,0,BrC(Cl)C(F)(F)F,BrC(Cl)C(F)(F)F,NCGC00090868-06,Inactive


In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
df_traning = pd.read_csv('/home/wzabsd/Desktop/NAIST-PBL1/descriptors_selected.csv')
df_traning_data = df_traning.loc[:,"MolLogP":"EState_VSA9"].values
df_traning_data=StandardScaler().fit_transform(df_traning_data)
df_traning.head()
reducer = umap.UMAP(random_state=2021)
embedding = reducer.fit_transform(df_traning_data)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
df_traning = pd.read_csv('/home/wzabsd/Desktop/NAIST-PBL1/descriptors_selected.csv')
df_traning.head()
df_traning_data = df_traning.loc[:,"FractionCSP3":"VSA_EState3"].values
df_traning_data=StandardScaler().fit_transform(df_traning_data)
reducer = umap.UMAP(random_state=24)
embedding = reducer.fit_transform(df_traning_data)
embedding.shape
import seaborn as sns
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=[sns.color_palette()[x] for x in df_traning.Class.map({"Active":1, "Inactive":0})])
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the descriptors dataset', fontsize=24)

In [ ]:
penguins = pd.read_csv("https://github.com/allisonhorst/palmerpenguins/raw/5b5891f01b52ae26ad8cb9755ec93672f49328a8/data/penguins_size.csv")
penguins = penguins.dropna()
penguins.species_short.value_counts()
reducer = umap.UMAP()
penguin_data = penguins[
    [
        "culmen_length_mm",
        "culmen_depth_mm",
        "flipper_length_mm",
        "body_mass_g",
    ]
].values
scaled_penguin_data = StandardScaler().fit_transform(penguin_data)
embedding = reducer.fit_transform(scaled_penguin_data)
penguins.head()
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=[sns.color_palette()[x] for x in penguins.species_short.map({"Adelie":0, "Chinstrap":1, "Gentoo":2})])
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the Penguin dataset', fontsize=24)

In [ ]:
import os
os.chdir('/home/wzabsd/Desktop/NAIST-PBL1/')

In [ ]:
from DeepPurpose import oneliner
from DeepPurpose.dataset import *
from DeepPurpose import dataset

X_drug, X_drug_names = dataset.read_file_repurposing_library("/home/wzabsd/Desktop/NAIST-PBL1/data/test.tab")

oneliner.repurpose(*load_SARS_CoV2_Protease_3CL(), 
                   X_repurpose=X_drug,
                   drug_names=X_drug_names,
                   pretrained_dir='/home/wzabsd/Desktop/NAIST-PBL1/save_folder/pretrained_models/DeepPurpose_BindingDB/',
                   finetune_epochs=100,
                   finetune_batch_size=128
                   )

In [8]:
import pandas as pd

df = pd.read_csv('/home/wzabsd/Desktop/NAIST-PBL1/datasets/3CL_enzymatic_activity-clean.tsv', sep='\t')

df=df.iloc[:,[0,1,3]]
df.head()
df.to_csv("/home/wzabsd/Desktop/NAIST-PBL1/data/test.csv",index=False)


,SAMPLE_ID,washed_SMILES,Class
0,NCGC00093656-03,Br/C=C/C=1C(=O)NC(=O)N([C@@H]2O[C@H](CO)[C@@H]...,Inactive
1,NCGC00262958-01,Br/C=C\C=1C(=O)NC(=O)N([C@@H]2[C@@H](O)[C@H](O...,Inactive
2,NCGC00162076-02,BrC(Br)(P(OP(=O)(OC[C@@H]1[C@@H](O)[C@@H](O)[C...,Inactive
3,NCGC00160445-02,BrC(C(=O)NC(=O)N)C(C)C,Inactive
4,NCGC00090868-06,BrC(Cl)C(F)(F)F,Inactive
